In [2]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Dropout, Flatten, Dense, Input
from keras import applications
from keras.layers import AveragePooling2D, GlobalAveragePooling2D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, CSVLogger, TensorBoard, LambdaCallback
from keras.applications.resnet50 import ResNet50, preprocess_input 
from keras.layers import Conv2D, Convolution2D, MaxPooling2D, ZeroPadding2D, BatchNormalization, Activation
from keras.optimizers import Adam
from keras import backend as K
import numpy as np
import pandas as pd
from keras import layers
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from keras.models import load_model
import random
import cv2
from PIL import Image
import os
from keras.utils import to_categorical
from collections import defaultdict
import jpeg4py as jpeg
from io import BytesIO

Using TensorFlow backend.


In [3]:
#parameters
classes = 10
batch_size=60
image_size = 224
train_total = 3500
validation_total = 4370

In [3]:
model_resnet = ResNet50(include_top=False, weights = None,input_shape=(image_size,image_size,3))

In [5]:
x = model_resnet.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = BatchNormalization()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
x = BatchNormalization()(x)
x = Dense(classes, activation='softmax')(x)

In [6]:
for layer in model_resnet.layers:
    layer.trainable = False

In [8]:
model = Model(inputs=model_resnet.input, outputs=x)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        input_1[0][0]                    
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 112, 112, 64) 0           bn_conv1[0][0]                   
__________________________________________________________________________________________________
max_poolin

In [5]:
train_datagen = ImageDataGenerator(horizontal_flip=True)
validation_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
                    

In [ ]:
train_generator = train_datagen.flow_from_directory(
                    'train',
                    target_size=(image_size,image_size),
                    batch_size=batch_size,
                    class_mode='categorical',
                    shuffle=True
                    )

In [10]:
validation_generator = validation_datagen.flow_from_directory(
                        '../validation_224/',
                        target_size=(image_size,image_size),
                        batch_size=batch_size,
                        class_mode='categorical',
                        shuffle=False)

Found 4370 images belonging to 10 classes.


In [17]:
# random cropping with the given crop_size - for training data

In [29]:
def random_crop(image, crop_size):
    h,w,d = image.shape
    rand_num_h = random.randint(0,h-crop_size)
    rand_num_w = random.randint(0,w-crop_size)
    image_crop = image[rand_num_h:rand_num_h+crop_size,rand_num_w:rand_num_w+crop_size,:]
    del image
    return image_crop

In [30]:
def train_gen(train_files, train_classes, batch_size, target_size, imggen):
    min_batch_size = batch_size //10
    train_data = list(zip(train_files, train_classes))
    train_data_dict = defaultdict(list)
    # creating a dictionary such that, each key is each class, -
    # useful to send equal no. of images for each batch in every epoch
    for i in range(len(train_data)):
        train_data_dict[train_data[i][0].split('/')[0]].append(train_data[i])
       
    while(True):
        for i in train_data_dict.keys():
            random.shuffle(train_data_dict[i])
        for start in range(0, len(train_data), min_batch_size):
            image_crop_list = []
            image_classes_list = []
            for j in train_data_dict.keys():
                end = min(start + min_batch_size, len(train_data_dict[j])) 
    
                for i in range(start,end):
                    image_classes_list.append(train_data_dict[j][i][1])
                
                x_batch = [train_data_dict[j][i][0] for i in range(start,end)]
                for i in x_batch:
                    image = mpimg.imread('train/' + i)
                    image = random_crop(image,image_size)
                    # changing orientation of the image for augmentation
                    if(np.random.rand()<0.5):
                        image = np.rot90(image,1,(0,1))
                    image_crop_list.append(image)
                    del image
                   
            
            if(len(image_classes_list)<batch_size-2*min_batch_size):
                break
            
            image_crop_list = [x/1. for x in image_crop_list]
            image_crop_list = [imggen.random_transform(x) for x in image_crop_list]
            image_crop_list = [preprocess_input(x) for x in image_crop_list]
            
            x_batch = np.array(image_crop_list, np.float32)
            y_batch = np.array(image_classes_list)         
            
            yield (x_batch, y_batch)

In [16]:
# cropping at the center - for validation set

In [31]:
def random_crop_valid(image, crop_size):
    h,w,d = image.shape
    image_crop = image[h//2-crop_size//2:h//2+crop_size//2,w//2-crop_size//2:w//2+crop_size//2,:]
    
    return image_crop

In [32]:
def valid_gen(valid_files, valid_classes, batch_size, target_size, imggen):

    valid_data = list(zip(valid_files, valid_classes))
  
    while(True):
        
        for start in range(0, len(valid_data), batch_size):
            image_crop_list = []
            image_classes_list = []
            end = min(start + batch_size, len(valid_data)) 
    
            x_batch = [valid_data[i][0] for i in range(start,end)]
            
            for index,i in enumerate(x_batch):
                image = mpimg.imread('../validation_224/' + i)
                if(len(image.shape)<3):
                        continue
                
                image = random_crop_valid(image,image_size)
                image_crop_list.append(image)
                image_classes_list.append(valid_data[start+index][1])
                
                del image
               
            
            
            image_crop_list = [x/1. for x in image_crop_list]
            image_crop_list = [preprocess_input(x) for x in image_crop_list]
            
            x_batch = np.array(image_crop_list, np.float32)
            y_batch = np.array(image_classes_list)         
            
            yield (x_batch, y_batch)

In [33]:
valid_aug_generator = valid_gen(validation_generator.filenames,to_categorical(validation_generator.classes),batch_size,image_size,validation_datagen) 

In [ ]:
train_crop_generator = train_gen(train_generator.filenames,to_categorical(train_generator.classes),batch_size,image_size,train_datagen) 

In [11]:
model.compile(loss='categorical_crossentropy',
             optimizer=Adam(lr=5e-5),
             metrics=['accuracy'])

In [15]:
# with pushbullet library, we can get live updates of the each epoch results to our phone

In [18]:
from pushbullet import Pushbullet
pb = Pushbullet('o.KiDDDXPuzV4qKbXh4Lywbgw1tK2oFfq1')

In [19]:
pushbullet_callback = LambdaCallback(
    on_epoch_end=lambda epoch, logs: pb.push_note("epoch: "+str(epoch),"train_loss: "+str(logs['loss'])+"    val_loss"+str(logs['val_loss'])))

In [20]:
callbacks = [ModelCheckpoint(filepath='resnet_unalt_2.hdf5', verbose=1, save_best_only=True, save_weights_only=True),
ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1),
#EarlyStopping(monitor='val_loss', patience=0, verbose=1),
CSVLogger('./3-metrics_unalt_1.csv'),
TensorBoard(log_dir='logs_unalt', write_graph=True),
pushbullet_callback]

In [156]:
model.fit_generator(
                    train_crop_generator,
                    steps_per_epoch = 5*int(np.ceil(train_total/batch_size)),
                    epochs=100,
                    validation_data=valid_aug_generator,
                    validation_steps= int(np.ceil(validation_total/batch_size)),
                    verbose=1,
                    callbacks=callbacks)

Epoch 1/100
294/295 [============================>.] - ETA: 1s - loss: 0.0469 - acc: 0.9893
Epoch 00001: val_loss improved from inf to 0.15885, saving model to resnet_unalt_2.hdf5
295/295 [==============================] - 371s 1s/step - loss: 0.0471 - acc: 0.9893 - val_loss: 0.1589 - val_acc: 0.9566
Epoch 2/100
294/295 [============================>.] - ETA: 1s - loss: 0.0430 - acc: 0.9899
Epoch 00002: val_loss did not improve
295/295 [==============================] - 330s 1s/step - loss: 0.0429 - acc: 0.9899 - val_loss: 0.1617 - val_acc: 0.9577
Epoch 3/100
294/295 [============================>.] - ETA: 1s - loss: 0.0429 - acc: 0.9901
Epoch 00003: val_loss did not improve
295/295 [==============================] - 330s 1s/step - loss: 0.0428 - acc: 0.9902 - val_loss: 0.1632 - val_acc: 0.9589
Epoch 4/100
104/295 [=========>....................] - ETA: 3:15 - loss: 0.0402 - acc: 0.9910

KeyboardInterrupt: 

In [12]:
model.load_weights('resnet_image_aug_full.hdf5')

In [25]:
model.evaluate_generator(valid_aug_generator,int(np.ceil(validation_total/batch_size)))

[0.13989693716752344, 0.96109839625980542]

In [7]:
# saving the validation predictions for ensembling later

In [36]:
resnet_preds = model.predict_generator(valid_aug_generator,int(np.ceil(validation_total/batch_size)))

In [37]:
f = defaultdict(list)
for i,j in enumerate(validation_generator.filenames):
    f[j.split('/')[-1][2:]].append(i)

In [40]:
result_filenames=[]
predicted_classes=[]
for i in validation_filenames:
    x = np.mean(resnet_preds[f[i.split('/')[-1]]],axis=0)
    result_filenames.append(i)
    predicted_classes.append(x)

In [39]:
validation_filenames = np.load('filenames_list.npy')

In [42]:
np.save('resnet_preds_224.npy',predicted_classes)

In [22]:
predictions_valid = model.predict_generator(valid_aug_generator,int(np.ceil(validation_total/batch_size)))
predictions_valid = np.argmax(predictions_valid, axis=1)

In [23]:
validation_generator.class_indices

{'HTC-1-M7': 0,
 'LG-Nexus-5x': 1,
 'Motorola-Droid-Maxx': 2,
 'Motorola-Nexus-6': 3,
 'Motorola-X': 4,
 'Samsung-Galaxy-Note3': 5,
 'Samsung-Galaxy-S4': 6,
 'Sony-NEX-7': 7,
 'iPhone-4s': 8,
 'iPhone-6': 9}

In [41]:
np.save('predictions.npy',predictions_valid)
np.save('correct_classes.npy',validation_generator.classes)

In [24]:
true_positive = np.zeros(10)
false_positive = np.zeros(10)
true_negative = np.zeros(10)
false_negative = np.zeros(10)
for i in range(len(predictions_valid)):
    if(predictions_valid[i]==validation_generator.classes[i]):
        true_positive[predictions_valid[i]] = true_positive[predictions_valid[i]] + 1
    else:
        false_positive[predictions_valid[i]] = false_positive[predictions_valid[i]] + 1
        false_negative[validation_generator.classes[i]] = false_negative[validation_generator.classes[i]] +1

In [25]:
print('true_positive')
print(true_positive)
print('false_positive')
print(false_positive)
print('false_negative')
print(false_negative)
print('Total')
print(true_positive+false_negative)


true_positive
[ 97.  98.  67.  68.  82.  73.  97.  80.  86.  89.]
false_positive
[ 6.  4.  3.  4.  1.  3.  4.  1.  4.  8.]
false_negative
[ 1.  1.  8.  9.  4.  3.  2.  6.  2.  2.]
Total
[ 98.  99.  75.  77.  86.  76.  99.  86.  88.  91.]


In [9]:
#predicting for 224 size crops of test images

In [13]:
filenames_list = []
images_list = []
for i in os.listdir('test_224/temp/'):
    img = Image.open('test_224/temp/'+i)
    filenames_list.append(i)
    images_list.append(np.array(img, np.float32))    

In [14]:
images_list = [preprocess_input(x) for x in images_list]
new_array_arr = np.array(images_list)
new_array_arr.shape

(13200, 224, 224, 3)

In [15]:
test_preds_224 = model.predict(new_array_arr)

In [13]:
# creating a dictionary such that each key is a test filename, and its values are the corresponding cropped filenames

In [16]:
f = defaultdict(list)
for i,j in enumerate(filenames_list):
    f[j[2:]].append(i)

In [19]:
result_filenames=[]
predicted_classes=[]
for i in test_filenames:
    x = np.mean(test_preds_224[f[i.split('/')[-1]]],axis=0)
    result_filenames.append(i)
    predicted_classes.append(x)

In [10]:
# saving test results for ensembling later

In [20]:
np.save('resnet_test_preds_224.npy',predicted_classes)

In [11]:
#predicting for 512 size(original size) test images

In [3]:
filenames_list = []
images_list = []
for i in os.listdir('test/temp/'):
    img = Image.open('test/temp/'+i)
    filenames_list.append(i)
    images_list.append(np.array(img, np.float32))
    

In [4]:
images_list = [preprocess_input(x) for x in images_list]

In [5]:
new_array_arr = np.array(images_list)


In [6]:
new_array_arr.shape

(2640, 512, 512, 3)

In [22]:
test_preds = model.predict(new_array_arr)

In [23]:
test_preds[0]

array([ 0.01716739,  0.01317034,  0.87034464,  0.00649183,  0.01907767,
        0.00433694,  0.00424909,  0.00927235,  0.0446626 ,  0.0112271 ], dtype=float32)

In [24]:
np.save('test_filenames.npy',filenames_list)
np.save('resnet_test_preds.npy',test_preds)

In [12]:
# for mapping class indices to class names

In [60]:
class_ids = {train_generator.class_indices[x]: x for x in train_generator.class_indices}

In [14]:
# submitting blanks for manip files, to know the score of only unalt images - 
# useful because we have seperate models for both manip and unalt images.

In [30]:
for index,i in enumerate(result_filenames):
    if(i.split('_')[-1]=='manip.tif'):
        predicted_classes[index]=''

In [65]:
submission = pd.DataFrame({'fname':result_filenames,'camera':predicted_classes})
submission.to_csv('submission_resnet_v2_ext_data_moto_x.csv', encoding="utf8", index=False)

In [66]:
from IPython.display import FileLink
FileLink('submission_resnet_v2_ext_data_moto_x.csv')

/home/janardhan/camera_model/submission_resnet_v2_ext_data_moto_x.csv

In [ ]:
# Private LB score - unalt+manip - 95.39